In [141]:
from rag.config import ROOT_DIR
from pathlib import Path
from rag.config import EFS_DIR, DOCS_DIR, MAX_CONTEXT_LENGTHS
import matplotlib.pyplot as plt
from dotenv import load_dotenv; load_dotenv()
import os
%load_ext autoreload
%autoreload 2
# langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from rag.data import extract_pages, get_sections, chunk_section


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load PDFs and split them into sections

In [83]:
# get path to docs
list_of_docs = [{"path": path} for path in DOCS_DIR.rglob("*.pdf") if not path.is_dir()]

In [144]:
# read in pdfs and split into sections
sections = list(map(get_sections,list_of_docs))
sections =[item for sublist in sections for item in sublist]
print(len(sections))

66


# Split Sectins further into chunks

In [145]:
chunks = list(map(chunk_section, sections))
chunks =[item for sublist in chunks for item in sublist]
print(len(chunks))

639


# Embedd and write into vector database

In [147]:
import chromadb
from chromadb.utils import embedding_functions


In [148]:
client = chromadb.PersistentClient(path="/Users/htr365/Documents/Side_Projects/09_founding_lab/semester_project/meta-studies/extraction/simple/chromadb/")
client.heartbeat()

1703613241998379000

In [149]:
embedding_model_name = "text-embedding-ada-002"

In [150]:
# define embedding function
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                model_name=embedding_model_name,
                api_key=os.environ["OPENAI_API_KEY"] 
            )

In [151]:
collection = client.get_or_create_collection(
        name="paper_collection1",
        metadata={"hnsw:space": "cosine"} ,
        embedding_function= openai_ef
    )

In [152]:
# function to add to collection
def add_to_collection(chunk):
    collection.upsert(
    documents= chunk.page_content,
    metadatas=chunk.metadata,
    ids=chunk.metadata['id']
    )

In [153]:
# embed and add chunks to database
list(map(add_to_collection,chunks))

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [154]:
collection.peek(5)

{'ids': ['3d755fb6-51b4-4abc-807c-7fc4f25b18be_0_0',
  '3d755fb6-51b4-4abc-807c-7fc4f25b18be_0_1',
  '3d755fb6-51b4-4abc-807c-7fc4f25b18be_0_10',
  '3d755fb6-51b4-4abc-807c-7fc4f25b18be_0_100',
  '3d755fb6-51b4-4abc-807c-7fc4f25b18be_0_101'],
 'embeddings': [[-0.013983952812850475,
   -0.007458552718162537,
   0.01621018722653389,
   -0.033140234649181366,
   -0.0057188901118934155,
   0.022502299398183823,
   0.002227901015430689,
   0.020262734964489937,
   -0.017903191968798637,
   0.0019446226069703698,
   -0.01229761354625225,
   0.019222935661673546,
   0.00047407468082383275,
   0.005022358614951372,
   -0.02362208254635334,
   -0.00016590530867688358,
   0.04556449130177498,
   0.012057660147547722,
   0.018036499619483948,
   -0.007971785962581635,
   -0.0121309794485569,
   0.03332686424255371,
   -0.015676958486437798,
   0.0007127783610485494,
   0.004712418653070927,
   -0.0029460948426276445,
   0.019222935661673546,
   -0.0377526730298996,
   -0.001872969907708466,
   -0

# Retrieval

In [155]:
query = "What is the main result of the study?"


In [156]:
context = collection.query(
    query_texts=[query],
    n_results=5,
    #where={"metadata_field": "is_equal_to_this"},
    #where_document={"$contains":"search_string"}
)

In [158]:
llm = 'gpt-3.5-turbo'
ai_client = get_client(llm='gpt-3.5-turbo')

In [159]:
system_content = 'Answer the query using the context provided. Be succinct. Answer in JSON format.'
user_content=f"query: {query} context: {context}"
assistant_content=''

In [160]:
import time

In [34]:
messages = [{"role": role, "content": content} for role, content in [
        ("system", system_content), 
        ("assistant", assistant_content), 
        ("user", user_content)] if content]

In [162]:
chat_completion = ai_client.chat.completions.create(
                model='gpt-4-0314'=,
                #response_format={ "type": "json_object" },
                temperature=0,
                stream=True,
                messages=messages,
            )

SyntaxError: invalid syntax (<ipython-input-162-5085035408ae>, line 2)

In [52]:
generate_response('gpt-4-0314', temperature=0.0, stream=True,
    system_content=system_content, assistant_content="", user_content=user_content, 
    max_retries=1, retry_interval=60)

<generator object response_stream at 0x7fe40881eea0>

In [53]:
from rag.embed import get_embedding_model
from rag.utils import get_num_tokens, trim


In [174]:
llm ='gpt-3.5-turbo'
agent = QueryAgent(
    embedding_model_name='',
    llm=llm,
    max_context_length=MAX_CONTEXT_LENGTHS[llm],
    system_content=system_content)
result = agent(query=query, stream=False)

TypeError: list indices must be integers or slices, not str

In [ ]:
result

{'question': 'What is the main result of the study?',
 'answer': '{\n  "main_result": "There was no significant difference in the incidence of recurrent myocardial infarction between the two groups."\n}',
 'llm': 'gpt-3.5-turbo'}

In [173]:
class QueryAgent:
    def __init__(self, embedding_model_name=embedding_model_name,
                 llm=llm, temperature=0.0, 
                 max_context_length=4096, system_content="", assistant_content=""):
        
        # Embedding model
      #  self.embedding_model = get_embedding_model(
       #     embedding_model_name=embedding_model_name, 
       #     model_kwargs={"device": "cuda"}, 
       #     encode_kwargs={"device": "cuda", "batch_size": 100})
        
        # Context length (restrict input length to 50% of total context length)
        max_context_length = int(0.5*max_context_length)
        
        # LLM
        self.llm = llm
        self.temperature = temperature
        self.context_length = max_context_length - get_num_tokens(system_content + assistant_content)
        self.system_content = system_content
        self.assistant_content = assistant_content

    def __call__(self, query, num_chunks=5, stream=True):
        # Get sources and context

        context_results = collection.query(
    query_texts=[query],
    n_results=num_chunks,
    #where={"metadata_field": "is_equal_to_this"},
    #where_document={"$contains":"search_string"}
)
       # context_results = semantic_search(
        ##   embedding_model=self.embedding_model, 
          #  k=num_chunks)
            
        # Generate response
        context = context_results['documents']
        #context = [item["text"] for item in context_results]
        #sources = [item["source"] for item in context_results]
        user_content = f"query: {query}, context: {context}"
        answer = generate_response(
            llm=self.llm,
            temperature=self.temperature,
            stream=stream,
            system_content=self.system_content,
            assistant_content=self.assistant_content,
            user_content=trim(user_content, self.context_length))

        # Result
        result = {
            "question": query,
            #"sources": sources,
            "sources": context
            'metadatas': context_results['metadatas'],
            "answer": answer,
            "llm": self.llm,
        }
        return result


In [172]:
context['metadatas']

[[{'chunk_id': '31',
   'content_font': 21,
   'heading': '3 JOURNAL\n',
   'heading_font': 38,
   'id': 'canmedaj00856-0002_0_31',
   'page': 1,
   'paper_id': 'canmedaj00856-0002',
   'section_id': 1},
  {'chunk_id': '65',
   'content_font': 21,
   'heading': '3 JOURNAL\n',
   'heading_font': 38,
   'id': 'canmedaj00856-0002_0_65',
   'page': 1,
   'paper_id': 'canmedaj00856-0002',
   'section_id': 1},
  {'chunk_id': '68',
   'content_font': 21,
   'heading': '3 JOURNAL\n',
   'heading_font': 38,
   'id': 'canmedaj00856-0002_0_68',
   'page': 1,
   'paper_id': 'canmedaj00856-0002',
   'section_id': 1},
  {'chunk_id': '84',
   'content_font': 10,
   'heading': 'A double-blind trial of long-term anticoagulant\ntreatment after myocardial infarction\n',
   'heading_font': 18,
   'id': '3d755fb6-51b4-4abc-807c-7fc4f25b18be_0_84',
   'page': 1,
   'paper_id': '3d755fb6-51b4-4abc-807c-7fc4f25b18be',
   'section_id': 0},
  {'chunk_id': '67',
   'content_font': 21,
   'heading': '3 JOURNAL\n'

In [47]:
def response_stream(chat_completion):
    for chunk in chat_completion:
        content = chunk.choices[0].delta.content
        if content is not None:
            yield content


In [44]:
def prepare_response(chat_completion, stream):
    if stream:
        return response_stream(chat_completion)
    else:
        return chat_completion.choices[0].message.content

In [51]:
def generate_response(
    llm, temperature=0.0, stream=True,
    system_content="", assistant_content="", user_content="", 
    max_retries=1, retry_interval=60):
    """Generate response from an LLM."""
    retry_count = 0
    client = get_client(llm=llm)
    messages = [{"role": role, "content": content} for role, content in [
        ("system", system_content), 
        ("assistant", assistant_content), 
        ("user", user_content)] if content]
    while retry_count <= max_retries:
        try:
            chat_completion = client.chat.completions.create(
                model=llm,
               # response_format={ "type": "json_object" },
                temperature=temperature,
                stream=stream,
                messages=messages,
            )
            return prepare_response(chat_completion, stream=stream)

        except Exception as e:
           print(f"Exception: {e}")
           time.sleep(retry_interval)  # default is per-minute rate limits
           retry_count += 1
    return chat_completion



In [171]:
# Generate response
#query = "What is the default batch size for map_batches?"
response = generate_response(
    llm ="gpt-3.5-turbo-1106",
    temperature=0.0,
    stream=True,
    system_content="You are a helpful assistant designed to output JSON. Answer the query using the context provided. Be succinct.",
    user_content=f"query: {query} context: {context}")
# Stream response
for content in response:
    print(content, end='', flush=True)

KeyboardInterrupt: 